ref: https://stackoverflow.com/questions/48467666/ranking-optimization

ref: https://developers.google.com/optimization/flow/assignment_min_cost_flow

ref: https://gitlab.com/darrylm/assign_to_workshops

In [216]:
from ortools.graph.python import min_cost_flow
import pandas as pd

# Arrange Data

In [217]:
data = pd.read_csv("data.csv")

In [218]:
# Shuffle to yield different outputs for equal penalties 
# (i.e. different people bumped down to lower ranked preferences in ties)
data = data.sample(frac=1)

In [219]:
data.head()

,Name,Bio,Chem,Soc,Phys,Lit,Gov,Tech,Env
16,Ali,1,8,6,3,4,2,5,7
1,Youssef,1,6,4,5,2,3,7,8
20,Selim,1,6,2,3,5,4,7,8
19,Murad,1,4,6,5,3,2,8,7
8,Hamza,1,6,2,5,3,4,7,8


In [220]:
capacities = {k:3 for k in data.columns[1:]}

In [221]:
capacities

{'Bio': 3,
 'Chem': 3,
 'Soc': 3,
 'Phys': 3,
 'Lit': 3,
 'Gov': 3,
 'Tech': 3,
 'Env': 3}

In [222]:
data = data.set_index("Name")
data = data.T

In [223]:
data.head()

Name,Ali,Youssef,Selim,Murad,Hamza,Bilal,Mohamed,Karim,Tareq,Mustafa,...,Abdel-Rahman,Yassin,Ibrahim,Mahmoud,Halim,Hassan,Abdallah,Hussein,Omar,Taha
Bio,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Chem,8,6,6,4,6,5,5,2,3,2,...,5,5,3,5,4,5,7,6,5,8
Soc,6,4,2,6,2,7,4,4,5,5,...,2,7,4,8,6,6,8,4,7,2
Phys,3,5,3,5,5,2,7,8,2,6,...,3,3,2,2,5,4,6,3,2,7
Lit,4,2,5,3,3,4,3,3,6,3,...,4,4,6,4,2,3,2,7,4,5


In [224]:
# mis-entered rankings?
q = data.nunique()
q[q < 8]

Series([], dtype: int64)

In [225]:
people_preferences = dict()
for name in data.columns:
    people_preferences[name] = data[name].sort_values().index.to_list()

# Optimizing

In [226]:
def penalty_func_square(val):
    return val ** 2

def penalty_func_ident(val):
    return val

def penalty_func_nonsense(val):
    return val ** 5

In [227]:
people_indexes = {person:i for i, person in enumerate(list(people_preferences))}
grp_indexes_offset = len(people_indexes)
grp_indexes = {grp:(i + grp_indexes_offset) for i, grp in enumerate(list(capacities))}
sink_index = len(people_indexes) + len(grp_indexes)

In [228]:
smcf = min_cost_flow.SimpleMinCostFlow()

In [229]:
# Create arc from each person to each workshop.
# Its capacity is 1 (the person can attend the workshop at most once),
# and the cost is the penalty based on the priority order).
# Also, track arc IDs so we can get extract results later.

person_grp_to_arc_index = dict()
for person, prefs in people_preferences.items():
    person_idx = people_indexes[person]
    for i in range(len(capacities)):
        grp = prefs[i]
        grp_idx = grp_indexes[grp]
        cost = penalty_func_square(i)  # or i + 1 to start w/ 1

        arc_index = smcf.add_arc_with_capacity_and_unit_cost(person_idx, grp_idx, 1, cost)
        person_grp_to_arc_index[(person, grp)] = arc_index


In [230]:
# create an arc from each workshop to the sink.
# Its capacity is the workshop capacity. Its cost is 0.
for grp, grp_idx in grp_indexes.items():
    grp_capacity = capacities[grp]
    smcf.add_arc_with_capacity_and_unit_cost(grp_idx, sink_index, grp_capacity, 0)

In [231]:
# Now add the node supplies.  They are 1 at each person (each person can
# go to at most 1 workshop), and -numPeople at the sink (each person needs to
# go to at least 1 workshop)

for person_idx in people_indexes.values():
    smcf.set_node_supply(person_idx, 1)
smcf.set_node_supply(sink_index, -len(people_preferences))

In [232]:
# Solve!
smcf.solve_max_flow_with_min_cost()

<Status.OPTIMAL: 1>

In [233]:
# Now extract results.
# Loop through each arc from person to workshop.
# If it's a 1, that's the workshop that person attended

assignments = dict()
for k, arc_idx in person_grp_to_arc_index.items():
    if smcf.flow(arc_idx) == 1:
        person = k[0]
        grp = k[1]
        assignments[person] = grp

In [234]:
# assignments
out_rows = []
for person in list(people_preferences):
    grp = assignments[person]
    rnk = people_preferences[person].index(grp) + 1
    out_rows.append([person, grp, rnk])
    # print("{} in {} (choice #{})".format(person, grp, rnk))

In [235]:
out = pd.DataFrame(out_rows, columns=["Name", "Group", "Rank of Group"])

In [236]:
out.head()

,Name,Group,Rank of Group
0,Ali,Gov,2
1,Youssef,Lit,2
2,Selim,Soc,2
3,Murad,Gov,2
4,Hamza,Soc,2


In [160]:
out.to_csv("optimized_placements.csv")